In [0]:
import numpy as np
from glob import glob
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
nd1 = np.zeros((0,79),dtype=object)
for x in glob('/content/drive/My Drive/Dataset/Data/*.csv'):
    print(x)
    nd1=np.vstack((nd1,pd.read_csv(x)))

/content/drive/My Drive/Dataset/Data/Friday-02-03-2018_TrafficForML_CICFlowMeter.csv
/content/drive/My Drive/Dataset/Data/Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv
/content/drive/My Drive/Dataset/Data/Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv
/content/drive/My Drive/Dataset/Data/Friday-16-02-2018_TrafficForML_CICFlowMeter.csv
/content/drive/My Drive/Dataset/Data/Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv
/content/drive/My Drive/Dataset/Data/Friday-23-02-2018_TrafficForML_CICFlowMeter.csv
/content/drive/My Drive/Dataset/Data/Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv
/content/drive/My Drive/Dataset/Data/Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv
/content/drive/My Drive/Dataset/Data/Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv


In [0]:
Dt1=nd1[:,:-1].astype(float)

In [0]:
LNMV1=nd1[~np.isnan(Dt1).any(axis=1),-1]
DtNMV1=Dt1[~np.isnan(Dt1).any(axis=1)]
LNMIV1=LNMV1[~np.isinf(DtNMV1).any(axis=1)]
DtNMIV1=DtNMV1[~np.isinf(DtNMV1).any(axis=1)]

In [0]:

del(DtNMV1)

In [0]:
np.save('/content/drive/My Drive/Result/NBx1', MinMaxScaler().fit_transform(DtNMIV1))
np.save('/content/drive/My Drive/Result/NBy1', (LNMIV1=='Benign').astype(int))

In [0]:
DtNMIV1.shape

(8247888, 78)

In [0]:

MCDt1=DtNMIV1[LNMIV1 !='Benign',:]

In [0]:
MCDt1.shape

(2170743, 78)

In [0]:
nd2 = np.zeros((0,79),dtype=object)
nd2=np.vstack((nd2,pd.read_csv('/content/drive/My Drive/Dataset/Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv')))

In [0]:
Dt2=nd2[:,:-1].astype(float)

In [0]:
LNMV2=nd2[~np.isnan(Dt2).any(axis=1),-1]
DtNMV2=Dt2[~np.isnan(Dt2).any(axis=1)]
LNMIV2=LNMV2[~np.isinf(DtNMV2).any(axis=1)]
DtNMIV2=DtNMV2[~np.isinf(DtNMV2).any(axis=1)]

In [0]:

del(DtNMV2)

In [0]:
np.save('/content/drive/My Drive/Result/NBx2', MinMaxScaler().fit_transform(DtNMIV2))
np.save('/content/drive/My Drive/Result/NBy2', (LNMIV2=='Benign').astype(int))

In [0]:
DtNMIV2.shape

(7889295, 78)

In [0]:

MCDt2=DtNMIV2[LNMIV2 !='Benign',:]

In [0]:
MCDt2.shape

(576191, 78)

In [3]:

import numpy as np
import os
import tensorflow as tf
import keras
from keras import models
from keras import layers

Using TensorFlow backend.


In [0]:
x1=np.load('/content/drive/My Drive/Result/NBx1.npy')
y1=np.load('/content/drive/My Drive/Result/NBy1.npy')
x2=np.load('/content/drive/My Drive/Result/NBx2.npy')
y2=np.load('/content/drive/My Drive/Result/NBy2.npy')
x=np.concatenate((x1,x2), axis=0)
y=np.concatenate((y1,y2), axis=0)
x=x.astype(np.float32)

In [0]:
x=np.delete(x,0,1)

In [8]:
(unique, counts) = np.unique(y, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[       0  2746934]
 [       1 13390249]]


In [9]:
x.shape

(16137183, 77)

In [0]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=123)

In [10]:
def create_model():
  model = models.Sequential()
  model.add(layers.Dense(256,input_dim=x.shape[1],activation='relu',use_bias=True))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(128,activation='relu',use_bias=True))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(64,activation='relu',use_bias=True))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(32,activation='relu',use_bias=True))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(1,activation='sigmoid',use_bias=True)) 
  return model
 
model = create_model()
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
model.summary()
history=model.fit(xtrain , ytrain ,batch_size=1024 , epochs=20 ,verbose=1 ,shuffle=True)
result = model.evaluate(xtest,ytest)
dict(zip(model.metrics_names, result))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               19968     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)               

{'accuracy': 0.9883638024330139, 'loss': 0.04276857356002982}

In [0]:
y_pred=model.predict(xtest)

In [12]:
from sklearn import metrics
print("The accuracy score is",metrics.accuracy_score(ytest, y_pred.round()))
print("The precision score is",metrics.precision_score(ytest, y_pred.round(),average='weighted'))
print("The recall score is",metrics.recall_score(ytest, y_pred.round(),average='weighted'))
print("The f1-score is",metrics.f1_score(ytest, y_pred.round(),average='weighted'))
print('\n')
print(metrics.confusion_matrix(ytest, y_pred.round()))
print('\n')
print("=== Classification Report ===")
print(metrics.classification_report(ytest, y_pred.round()))
print('\n')

The accuracy score is 0.9883638317339735
The precision score is 0.9884547367456414
The recall score is 0.9883638317339735
The f1-score is 0.9882185494582253


[[ 514446   35378]
 [   2177 2675436]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       1.00      0.94      0.96    549824
           1       0.99      1.00      0.99   2677613

    accuracy                           0.99   3227437
   macro avg       0.99      0.97      0.98   3227437
weighted avg       0.99      0.99      0.99   3227437





In [0]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc_history=rfc.fit(xtrain,ytrain)
rfc_predict = rfc.predict(xtest)

In [15]:
from sklearn import metrics
print("The accuracy score is",metrics.accuracy_score(ytest, rfc_predict))
print("The precision score is",metrics.precision_score(ytest, rfc_predict,average='weighted'))
print("The recall score is",metrics.recall_score(ytest, rfc_predict,average='weighted'))
print("The f1-score is",metrics.f1_score(ytest, rfc_predict,average='weighted'))
print('\n')
print(metrics.confusion_matrix(ytest, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(metrics.classification_report(ytest, rfc_predict))
print('\n')

The accuracy score is 0.986718253524391
The precision score is 0.9867128304513948
The recall score is 0.986718253524391
The f1-score is 0.9865768177786151


[[ 514151   35673]
 [   7193 2670420]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.99      0.94      0.96    549824
           1       0.99      1.00      0.99   2677613

    accuracy                           0.99   3227437
   macro avg       0.99      0.97      0.98   3227437
weighted avg       0.99      0.99      0.99   3227437



